In [55]:
import numpy as np
from scipy.fft import fft, fftfreq
from scipy.signal import find_peaks
from scipy.io import wavfile
import matplotlib.pyplot as plt
import soundfile as sf
import librosa

#### Lectura de archivos

In [56]:
def read_file(file):
    if file.endswith('.wav'):
        sr, data = wavfile.read(file)
    
    elif file.endswith('.mp3'):
        data, sr = librosa.load(file)

    else:
        raise ValueError('File format not supported')
    
    data = data[:,0] if len(data.shape) > 1 else data

    return data, sr

In [57]:
carpeta_archivos_wav = "piano-88-notes"
carpeta_archivos_mp3 = "gleitz midi-js-soundfonts gh-pages FluidR3_GM-acoustic_grand_piano-mp3"

#### Diccionario Notas

In [58]:
notas_basicas = {
    "Do": 261.63,
    "Re": 293.66,
    "Mi": 329.63,
    "Fa": 349.23,
    "Sol": 392.00,
    "La": 440.00,
    "Si": 493.88,
    r"Do sostenido / Re bemol": 277.18,
    r"Re sostenido / Mi bemol": 311.13,
    r"Fa sostenido / Sol bemol": 369.99,
    r"Sol sostenido / La bemol": 415.30,
    r"La sostenido / Si bemol": 466.16
}

In [59]:
archivos_mp3 = ["A4.mp3", "B4.mp3", "C4.mp3", "D4.mp3", "E4.mp3", "F4.mp3", "G4.mp3", "Bb4.mp3", "Db4.mp3", "Eb4.mp3", "Gb4.mp3", "Ab4.mp3"]
archivos_wav = ["4-a.wav", "4-b.wav", "4-c.wav", "4-d.wav", "4-e.wav", "4-f.wav", "4-g.wav", "4-as.wav", "4-cs.wav", "4-ds.wav", "4-fs.wav", "4-gs.wav"]

In [60]:
notas_correctas_mp3 = {
    "A4.mp3": "La",
    "B4.mp3": "Si",
    "C4.mp3": "Do",
    "D4.mp3": "Re",
    "E4.mp3": "Mi",
    "F4.mp3": "Fa",
    "G4.mp3": "Sol",
    "Bb4.mp3": r"La sostenido / Si bemol",
    "Db4.mp3": r"Do sostenido / Re bemol",
    "Eb4.mp3": r"Re sostenido / Mi bemol",
    "Gb4.mp3": r"Fa sostenido / Sol bemol",
    "Ab4.mp3": r"Sol sostenido / La bemol"
}

notas_correctas_wav = {
    "4-a.wav": "La",
    "4-b.wav": "Si",
    "4-c.wav": "Do",
    "4-d.wav": "Re",
    "4-e.wav": "Mi",
    "4-f.wav": "Fa",
    "4-g.wav": "Sol",
    "4-as.wav": r"La sostenido / Si bemol",
    "4-cs.wav": r"Do sostenido / Re bemol",
    "4-ds.wav": r"Re sostenido / Mi bemol",
    "4-fs.wav": r"Fa sostenido / Sol bemol",
    "4-gs.wav": r"Sol sostenido / La bemol"
}

#### Funciones para detectar notas

In [61]:
def detectar_picos(Y, umbral=0.01):
    peaks, _ = find_peaks(Y, height=umbral)
    return peaks

def calcular_umbral_maximo(yf, porcentaje=0.1):
    umbral = porcentaje * np.max(yf)
    return umbral

def apply_fft(data, sr):
    N = len(data)
    T = 1.0 / sr
    yf = fft(data)
    xf = fftfreq(N, T)[:N//2]
    yf = 2.0/N * np.abs(yf[0:N//2])
    return xf, yf

def get_nearest_note(fundamental_freq, dict_notas):
    nearest_note = min(dict_notas, key=lambda nota: abs(dict_notas[nota] - fundamental_freq))
    frecuencia_mas_cercana = dict_notas[nearest_note]
    return nearest_note, frecuencia_mas_cercana

def plot_fft(xf, yf, peaks):
    plt.figure()
    plt.plot(xf, yf)
    plt.plot(xf[peaks], yf[peaks], "x")
    plt.grid()
    plt.title("FFT")
    plt.xlabel("Frecuencia (Hz)")
    plt.ylabel("Amplitud")
    plt.show()

def plot_signal(data):
    plt.figure()
    plt.plot(data)
    plt.grid()
    plt.title("Señal")
    plt.xlabel("Tiempo")
    plt.ylabel("Amplitud")
    plt.show()

#### Para archivos mp3

In [62]:
correctas = 0

for archivo in archivos_mp3:
    data, sr = read_file(f"{carpeta_archivos_mp3}/{archivo}")
    #plot_signal(data)
    xf, yf = apply_fft(data, sr)
    umbral = calcular_umbral_maximo(yf)
    peaks = detectar_picos(yf, umbral=umbral)
    #plot_fft(xf, yf, peaks)
    fundamental_freq = xf[peaks[0]]
    nearest_note, frecuencia_mas_cercana = get_nearest_note(fundamental_freq, notas_basicas)

    if nearest_note == notas_correctas_mp3[archivo]:
        correctas += 1
    
    else:
        print("Fallo archivo:", archivo, "Nota detectada:", nearest_note, "Nota correcta:", notas_correctas_mp3[archivo])

    print(f"Archivo: {archivo}", f"Fundamental: {fundamental_freq} Hz", f"Nota: {nearest_note}", f"Frecuencia más cercana: {frecuencia_mas_cercana} Hz")
    print("------------------------")

print(f"Correctas: {correctas}/{len(archivos_mp3)}")

Archivo: A4.mp3 Fundamental: 440.0923295454545 Hz Nota: La Frecuencia más cercana: 440.0 Hz
------------------------
Archivo: B4.mp3 Fundamental: 489.6306818181818 Hz Nota: Si Frecuencia más cercana: 493.88 Hz
------------------------
Archivo: C4.mp3 Fundamental: 261.43465909090907 Hz Nota: Do Frecuencia más cercana: 261.63 Hz
------------------------
Archivo: D4.mp3 Fundamental: 293.3948863636363 Hz Nota: Re Frecuencia más cercana: 293.66 Hz
------------------------
Archivo: E4.mp3 Fundamental: 327.5923295454545 Hz Nota: Mi Frecuencia más cercana: 329.63 Hz
------------------------
Archivo: F4.mp3 Fundamental: 349.0056818181818 Hz Nota: Fa Frecuencia más cercana: 349.23 Hz
------------------------
Archivo: G4.mp3 Fundamental: 392.1519886363636 Hz Nota: Sol Frecuencia más cercana: 392.0 Hz
------------------------
Archivo: Bb4.mp3 Fundamental: 466.2997159090909 Hz Nota: La sostenido / Si bemol Frecuencia más cercana: 466.16 Hz
------------------------
Archivo: Db4.mp3 Fundamental: 277.

#### Para archivos .wav

In [63]:
correctas = 0

for archivo in archivos_wav:
    data, sr = read_file(f"{carpeta_archivos_wav}/{archivo}")
    #plot_signal(data)
    xf, yf = apply_fft(data, sr)
    umbral = calcular_umbral_maximo(yf)
    peaks = detectar_picos(yf, umbral=umbral)
    #plot_fft(xf, yf, peaks)
    fundamental_freq = xf[peaks[0]]
    nearest_note, frecuencia_mas_cercana = get_nearest_note(fundamental_freq, notas_basicas)

    if nearest_note == notas_correctas_wav[archivo]:
        correctas += 1

    print(f"Archivo: {archivo}", f"Fundamental: {fundamental_freq} Hz", f"Nota: {nearest_note}", f"Frecuencia más cercana: {frecuencia_mas_cercana} Hz")
    print("------------------------")

print(f"Correctas: {correctas}/{len(archivos_wav)}")

Archivo: 4-a.wav Fundamental: 439.8 Hz Nota: La Frecuencia más cercana: 440.0 Hz
------------------------
Archivo: 4-b.wav Fundamental: 493.75 Hz Nota: Si Frecuencia más cercana: 493.88 Hz
------------------------
Archivo: 4-c.wav Fundamental: 261.5 Hz Nota: Do Frecuencia más cercana: 261.63 Hz
------------------------
Archivo: 4-d.wav Fundamental: 293.5 Hz Nota: Re Frecuencia más cercana: 293.66 Hz
------------------------
Archivo: 4-e.wav Fundamental: 329.5 Hz Nota: Mi Frecuencia más cercana: 329.63 Hz
------------------------
Archivo: 4-f.wav Fundamental: 349.0 Hz Nota: Fa Frecuencia más cercana: 349.23 Hz
------------------------
Archivo: 4-g.wav Fundamental: 391.8 Hz Nota: Sol Frecuencia más cercana: 392.0 Hz
------------------------
Archivo: 4-as.wav Fundamental: 466.0 Hz Nota: La sostenido / Si bemol Frecuencia más cercana: 466.16 Hz
------------------------
Archivo: 4-cs.wav Fundamental: 277.0 Hz Nota: Do sostenido / Re bemol Frecuencia más cercana: 277.18 Hz
------------------

C:\Users\Alejandro\AppData\Local\Temp\ipykernel_12352\1525891727.py:3: WavFileWarning: Chunk (non-data) not understood, skipping it.
  sr, data = wavfile.read(file)


Archivo: 4-fs.wav Fundamental: 369.8 Hz Nota: Fa sostenido / Sol bemol Frecuencia más cercana: 369.99 Hz
------------------------
Archivo: 4-gs.wav Fundamental: 415.20000000000005 Hz Nota: Sol sostenido / La bemol Frecuencia más cercana: 415.3 Hz
------------------------
Correctas: 12/12


### Detección de Octavas

#### Diccionario con octavas

In [64]:
notas_con_octavas = {
    "Do4": 261.63,
    "Re4": 293.66,
    "Mi4": 329.63,
    "Fa4": 349.23,
    "Sol4": 392.00,
    "La4": 440.00,
    "Si4": 493.88,
    r"Do sostenido / Re bemol 4": 277.18,
    r"Re sostenido / Mi bemol 4": 311.13,
    r"Fa sostenido / Sol bemol 4": 369.99,
    r"Sol sostenido / La bemol 4": 415.30,
    r"La sostenido / Si bemol 4": 466.16,
    "Do1": 32.70,
    "Re1": 36.71,
    "Mi1": 41.20,
    "Fa1": 43.65,
    "Sol1": 49.00,
    "La1": 55.00,
    "Si1": 61.74,
    r"Do sostenido / Re bemol 1": 34.65,
    r"Re sostenido / Mi bemol 1": 38.89,
    r"Fa sostenido / Sol bemol 1": 46.25,
    r"Sol sostenido / La bemol 1": 51.91,
    r"La sostenido / Si bemol 1": 58.27,
    "Do2": 65.41,
    "Re2": 73.42,
    "Mi2": 82.41,
    "Fa2": 87.31,
    "Sol2": 98.00,
    "La2": 110.00,
    "Si2": 123.47,
    r"Do sostenido / Re bemol 2": 69.30,
    r"Re sostenido / Mi bemol 2": 77.78,
    r"Fa sostenido / Sol bemol 2": 92.50,
    r"Sol sostenido / La bemol 2": 103.83,
    r"La sostenido / Si bemol 2": 116.54,
    "Do3": 130.81,
    "Re3": 146.83,
    "Mi3": 164.81,
    "Fa3": 174.61,
    "Sol3": 196.00,
    "La3": 220.00,
    "Si3": 246.94,
    r"Do sostenido / Re bemol 3": 138.59,
    r"Re sostenido / Mi bemol 3": 155.56,
    r"Fa sostenido / Sol bemol 3": 185.00,
    r"Sol sostenido / La bemol 3": 207.65,
    r"La sostenido / Si bemol 3": 233.08,
    "Do5": 523.25,
    "Re5": 587.33,
    "Mi5": 659.26,
    "Fa5": 698.46,
    "Sol5": 783.99,
    "La5": 880.00,
    "Si5": 987.77,
    r"Do sostenido / Re bemol 5": 554.37,
    r"Re sostenido / Mi bemol 5": 622.25,
    r"Fa sostenido / Sol bemol 5": 739.99,
    r"Sol sostenido / La bemol 5": 830.61,
    r"La sostenido / Si bemol 5": 932.33,
    "Do6": 1046.50,
    "Re6": 1174.66,
    "Mi6": 1318.51,
    "Fa6": 1396.91,
    "Sol6": 1567.98,
    "La6": 1760.00,
    "Si6": 1975.53,
    r"Do sostenido / Re bemol 6": 1108.73,
    r"Re sostenido / Mi bemol 6": 1244.51,
    r"Fa sostenido / Sol bemol 6": 1479.98,
    r"Sol sostenido / La bemol 6": 1661.22,
    r"La sostenido / Si bemol 6": 1864.66,
    "Do7": 2093.00,
    "Re7": 2349.32,
    "Mi7": 2637.02,
    "Fa7": 2793.83,
    "Sol7": 3135.96,
    "La7": 3520.00,
    "Si7": 3951.07,
    r"Do sostenido / Re bemol 7": 2217.46,
    r"Re sostenido / Mi bemol 7": 2489.02,
    r"Fa sostenido / Sol bemol 7": 2959.96,
    r"Sol sostenido / La bemol 7": 3322.44,
    r"La sostenido / Si bemol 7": 3729.31,
    "Do0": 16.35,
    "Re0": 18.35,
    "Mi0": 20.60,
    "Fa0": 21.83,
    "Sol0": 24.50,
    "La0": 27.50,
    "Si0": 30.87,
    r"Do sostenido / Re bemol 0": 17.32,
    r"Re sostenido / Mi bemol 0": 19.45,
    r"Fa sostenido / Sol bemol 0": 23.12,
    r"Sol sostenido / La bemol 0": 25.96,
    r"La sostenido / Si bemol 0": 29.14,
    "Do8": 4186.01,
}

In [65]:
notas_correctas_mp3_completo = {
    "A4.mp3": "La4", "B4.mp3": "Si4", "C4.mp3": "Do4", "D4.mp3": "Re4", "E4.mp3": "Mi4", 
    "F4.mp3": "Fa4", "G4.mp3": "Sol4", "Bb4.mp3": r"La sostenido / Si bemol 4", 
    "Db4.mp3": r"Do sostenido / Re bemol 4", "Eb4.mp3": r"Re sostenido / Mi bemol 4", 
    "Gb4.mp3": r"Fa sostenido / Sol bemol 4", "Ab4.mp3": r"Sol sostenido / La bemol 4",

    "A3.mp3": "La3", "B3.mp3": "Si3", "C3.mp3": "Do3", "D3.mp3": "Re3", "E3.mp3": "Mi3", 
    "F3.mp3": "Fa3", "G3.mp3": "Sol3", "Bb3.mp3": r"La sostenido / Si bemol 3", 
    "Db3.mp3": r"Do sostenido / Re bemol 3", "Eb3.mp3": r"Re sostenido / Mi bemol 3", 
    "Gb3.mp3": r"Fa sostenido / Sol bemol 3", "Ab3.mp3": r"Sol sostenido / La bemol 3",

    "A5.mp3": "La5", "B5.mp3": "Si5", "C5.mp3": "Do5", "D5.mp3": "Re5", "E5.mp3": "Mi5", 
    "F5.mp3": "Fa5", "G5.mp3": "Sol5", "Bb5.mp3": r"La sostenido / Si bemol 5", 
    "Db5.mp3": r"Do sostenido / Re bemol 5", "Eb5.mp3": r"Re sostenido / Mi bemol 5", 
    "Gb5.mp3": r"Fa sostenido / Sol bemol 5", "Ab5.mp3": r"Sol sostenido / La bemol 5",

    "A2.mp3": "La2", "B2.mp3": "Si2", "C2.mp3": "Do2", "D2.mp3": "Re2", "E2.mp3": "Mi2", 
    "F2.mp3": "Fa2", "G2.mp3": "Sol2", "Bb2.mp3": r"La sostenido / Si bemol 2", 
    "Db2.mp3": r"Do sostenido / Re bemol 2", "Eb2.mp3": r"Re sostenido / Mi bemol 2", 
    "Gb2.mp3": r"Fa sostenido / Sol bemol 2", "Ab2.mp3": r"Sol sostenido / La bemol 2",

    "A1.mp3": "La1", "B1.mp3": "Si1", "C1.mp3": "Do1", "D1.mp3": "Re1", "E1.mp3": "Mi1", 
    "F1.mp3": "Fa1", "G1.mp3": "Sol1", "Bb1.mp3": r"La sostenido / Si bemol 1", 
    "Db1.mp3": r"Do sostenido / Re bemol 1", "Eb1.mp3": r"Re sostenido / Mi bemol 1", 
    "Gb1.mp3": r"Fa sostenido / Sol bemol 1", "Ab1.mp3": r"Sol sostenido / La bemol 1",

    "A6.mp3": "La6", "B6.mp3": "Si6", "C6.mp3": "Do6", "D6.mp3": "Re6", "E6.mp3": "Mi6", 
    "F6.mp3": "Fa6", "G6.mp3": "Sol6", "Bb6.mp3": r"La sostenido / Si bemol 6", 
    "Db6.mp3": r"Do sostenido / Re bemol 6", "Eb6.mp3": r"Re sostenido / Mi bemol 6", 
    "Gb6.mp3": r"Fa sostenido / Sol bemol 6", "Ab6.mp3": r"Sol sostenido / La bemol 6",

    "A7.mp3": "La7", "B7.mp3": "Si7", "C7.mp3": "Do7", "D7.mp3": "Re7", "E7.mp3": "Mi7", 
    "F7.mp3": "Fa7", "G7.mp3": "Sol7", "Bb7.mp3": r"La sostenido / Si bemol 7", 
    "Db7.mp3": r"Do sostenido / Re bemol 7", "Eb7.mp3": r"Re sostenido / Mi bemol 7", 
    "Gb7.mp3": r"Fa sostenido / Sol bemol 7", "Ab7.mp3": r"Sol sostenido / La bemol 7",

    "A0.mp3": "La0", "B0.mp3": "Si0", "C0.mp3": "Do0", "D0.mp3": "Re0", "E0.mp3": "Mi0", 
    "F0.mp3": "Fa0", "G0.mp3": "Sol0", "Bb0.mp3": r"La sostenido / Si bemol 0", 
    "Db0.mp3": r"Do sostenido / Re bemol 0", "Eb0.mp3": r"Re sostenido / Mi bemol 0", 
    "Gb0.mp3": r"Fa sostenido / Sol bemol 0", "Ab0.mp3": r"Sol sostenido / La bemol 0",
    "C8.mp3": "Do8"
}


notas_correctas_wav_completo = {
    "4-a.wav": "La4", "4-b.wav": "Si4", "4-c.wav": "Do4", "4-d.wav": "Re4", "4-e.wav": "Mi4", 
    "4-f.wav": "Fa4", "4-g.wav": "Sol4", "4-as.wav": r"La sostenido / Si bemol 4", 
    "4-cs.wav": r"Do sostenido / Re bemol 4", "4-ds.wav": r"Re sostenido / Mi bemol 4", 
    "4-fs.wav": r"Fa sostenido / Sol bemol 4", "4-gs.wav": r"Sol sostenido / La bemol 4",

    "3-a.wav": "La3", "3-b.wav": "Si3", "3-c.wav": "Do3", "3-d.wav": "Re3", "3-e.wav": "Mi3", 
    "3-f.wav": "Fa3", "3-g.wav": "Sol3", "3-as.wav": r"La sostenido / Si bemol 3", 
    "3-cs.wav": r"Do sostenido / Re bemol 3", "3-ds.wav": r"Re sostenido / Mi bemol 3", 
    "3-fs.wav": r"Fa sostenido / Sol bemol 3", "3-gs.wav": r"Sol sostenido / La bemol 3",

    "5-a.wav": "La5", "5-b.wav": "Si5", "5-c.wav": "Do5", "5-d.wav": "Re5", "5-e.wav": "Mi5", 
    "5-f.wav": "Fa5", "5-g.wav": "Sol5", "5-as.wav": r"La sostenido / Si bemol 5", 
    "5-cs.wav": r"Do sostenido / Re bemol 5", "5-ds.wav": r"Re sostenido / Mi bemol 5", 
    "5-fs.wav": r"Fa sostenido / Sol bemol 5", "5-gs.wav": r"Sol sostenido / La bemol 5",

    "2-a.wav": "La2", "2-b.wav": "Si2", "2-c.wav": "Do2", "2-d.wav": "Re2", "2-e.wav": "Mi2", 
    "2-f.wav": "Fa2", "2-g.wav": "Sol2", "2-as.wav": r"La sostenido / Si bemol 2", 
    "2-cs.wav": r"Do sostenido / Re bemol 2", "2-ds.wav": r"Re sostenido / Mi bemol 2", 
    "2-fs.wav": r"Fa sostenido / Sol bemol 2", "2-gs.wav": r"Sol sostenido / La bemol 2",

    "1-a.wav": "La1", "1-b.wav": "Si1", "1-c.wav": "Do1", "1-d.wav": "Re1", "1-e.wav": "Mi1", 
    "1-f.wav": "Fa1", "1-g.wav": "Sol1", "1-as.wav": r"La sostenido / Si bemol 1", 
    "1-cs.wav": r"Do sostenido / Re bemol 1", "1-ds.wav": r"Re sostenido / Mi bemol 1", 
    "1-fs.wav": r"Fa sostenido / Sol bemol 1", "1-gs.wav": r"Sol sostenido / La bemol 1",

    "6-a.wav": "La6", "6-b.wav": "Si6", "6-c.wav": "Do6", "6-d.wav": "Re6", "6-e.wav": "Mi6", 
    "6-f.wav": "Fa6", "6-g.wav": "Sol6", "6-as.wav": r"La sostenido / Si bemol 6", 
    "6-cs.wav": r"Do sostenido / Re bemol 6", "6-ds.wav": r"Re sostenido / Mi bemol 6", 
    "6-fs.wav": r"Fa sostenido / Sol bemol 6", "6-gs.wav": r"Sol sostenido / La bemol 6",

    "7-a.wav": "La7", "7-b.wav": "Si7", "7-c.wav": "Do7", "7-d.wav": "Re7", "7-e.wav": "Mi7", 
    "7-f.wav": "Fa7", "7-g.wav": "Sol7", "7-as.wav": r"La sostenido / Si bemol 7", 
    "7-cs.wav": r"Do sostenido / Re bemol 7", "7-ds.wav": r"Re sostenido / Mi bemol 7", 
    "7-fs.wav": r"Fa sostenido / Sol bemol 7", "7-gs.wav": r"Sol sostenido / La bemol 7",

    "0-a.wav": "La0", "0-b.wav": "Si0", "0-c.wav": "Do0", "0-d.wav": "Re0", "0-e.wav": "Mi0", 
    "0-f.wav": "Fa0", "0-g.wav": "Sol0", "0-as.wav": r"La sostenido / Si bemol 0", 
    "0-cs.wav": r"Do sostenido / Re bemol 0", "0-ds.wav": r"Re sostenido / Mi bemol 0", 
    "0-fs.wav": r"Fa sostenido / Sol bemol 0", "0-gs.wav": r"Sol sostenido / La bemol 0",
    "8-c.wav": "Do8"
}


#### Para archivos mp3

In [66]:
#obtener el nombre de los archivos en el directorio carpeta_archivos_mp3
import os
archivos_mp3 = os.listdir(carpeta_archivos_mp3)

In [67]:
for porcentaje in [0.01, 0.05, 0.1, 0.5, 0.6, 0.7, 0.8, 0.9]:
    correctas = 0
    for archivo in archivos_mp3:
        data, sr = read_file(f"{carpeta_archivos_mp3}/{archivo}")
        #plot_signal(data)
        xf, yf = apply_fft(data, sr)
        umbral = calcular_umbral_maximo(yf, porcentaje=porcentaje)
        peaks = detectar_picos(yf, umbral=umbral)
        #plot_fft(xf, yf, peaks)
        fundamental_freq = xf[peaks[0]]
        nearest_note, frecuencia_mas_cercana = get_nearest_note(fundamental_freq, notas_con_octavas)

        if nearest_note == notas_correctas_mp3_completo[archivo]:
            correctas += 1

    print(f"Correctas: {correctas}/{len(archivos_mp3)} para porcentaje {porcentaje}")

Correctas: 6/88 para porcentaje 0.01
Correctas: 46/88 para porcentaje 0.05
Correctas: 66/88 para porcentaje 0.1
Correctas: 75/88 para porcentaje 0.5
Correctas: 71/88 para porcentaje 0.6
Correctas: 70/88 para porcentaje 0.7
Correctas: 70/88 para porcentaje 0.8
Correctas: 70/88 para porcentaje 0.9


#### Para archivos wav

In [68]:
archivos_wav = os.listdir(carpeta_archivos_wav)

for porcentaje in [0.01, 0.05, 0.1, 0.5, 0.6, 0.7, 0.8, 0.9]:
    correctas = 0
    for archivo in archivos_wav:
        data, sr = read_file(f"{carpeta_archivos_wav}/{archivo}")
        #plot_signal(data)
        xf, yf = apply_fft(data, sr)
        umbral = calcular_umbral_maximo(yf, porcentaje=porcentaje)
        peaks = detectar_picos(yf, umbral=umbral)
        #plot_fft(xf, yf, peaks)
        fundamental_freq = xf[peaks[0]]
        nearest_note, frecuencia_mas_cercana = get_nearest_note(fundamental_freq, notas_con_octavas)

        if nearest_note == notas_correctas_wav_completo[archivo]:
            correctas += 1

    print(f"Correctas: {correctas}/{len(archivos_wav)} para porcentaje {porcentaje}")

C:\Users\Alejandro\AppData\Local\Temp\ipykernel_12352\1525891727.py:3: WavFileWarning: Chunk (non-data) not understood, skipping it.
  sr, data = wavfile.read(file)


Correctas: 44/88 para porcentaje 0.01
Correctas: 83/88 para porcentaje 0.05
Correctas: 78/88 para porcentaje 0.1
Correctas: 64/88 para porcentaje 0.5
Correctas: 64/88 para porcentaje 0.6
Correctas: 60/88 para porcentaje 0.7
Correctas: 60/88 para porcentaje 0.8
Correctas: 58/88 para porcentaje 0.9
